## $\S$ 11.9.1. Bayes, Boosting, and Bagging
Let us first review briefly the Bayesian approach to inference and its application to neural networks.

### Basics
Given training data $\mathbf{X}_{\text{tr}}$, $\mathbf{y}_{\text{tr}}$, we assume a sampling model with parameters $\theta$; Neal and Zhang (2006) use a two-hidden-layer neural network with output nodes the class probabilities $\text{Pr}(Y \mid X, \theta)$ for the binary outcomes.

Given a prior distribution $\text{Pr}(\theta)$, the posterior distrbution for the parameter is

\begin{equation}
\text{Pr}(\theta \mid \mathbf{X}_{\text{tr}}, \mathbf{y}_{\text{tr}}) = \frac{\text{Pr}(\theta) \text{Pr}(\mathbf{y}_{\text{tr}} \mid \mathbf{X}_{\text{tr}}, \theta)}{\int \text{Pr}(\theta) \text{Pr}(\mathbf{y}_{\text{tr}} \mid \mathbf{X}_{\text{tr}}, \theta) d\theta}
\end{equation}

For a test case with features $X_{\text{new}}$, the predictive distribution for the label $Y_{\text{new}}$ is (c.f. equation 8.24)

\begin{equation}
\text{Pr}(Y_{\text{new}} \mid X_{\text{new}}, \mathbf{X}_{\text{tr}}, \mathbf{y}_{\text{tr}}) = \int \text{Pr}( Y_{\text{new}} \mid X_{\text{new}}, \theta) \text{Pr}(\theta \mid \mathbf{X}_{\text{tr}}, \mathbf{y}_{\text{tr}}) d\theta.
\end{equation}

### Markov Chain Monte Carlo
Since this integral is intractable, sophisticated Markov Chain Monte Carlo (MCMC) methods are used to sample from the posterior distribution $\text{Pr}(Y_{\text{new}} \mid X_{\text{new}}, \mathbf{X}_{\text{tr}}, \mathbf{y}_{\text{tr}})$. A few hundred values $\theta$ are generated and then a simple average of these values estimates the integral.

####  Hybrid Monte Carlo
Neal and Zhang (2006) use diffuse Gaussian priors for all of the parameters. The particular MCMC approach that was used is called _hybrid Monte Carlo_, and may be important for the success of the method. It includes an auxiliary momentum vector and implements Hamiltonian dynamics in which the potential function is the target density. This is done to avoid radom walk behavior; the successive candidates move across the sample space in larger steps. They tend to be less correlated and hence converge to the target distribution more rapidly.

### Preprocessing
Neal and Zhang (2006) also tried different forms of pre-processing of the features:
1. Univariate screening using t-tests, and
2. automatic relevance determination

In the latter method (ARD), the weights (coefficients) for the $j$th feature to each of the first hidden layer units all share a common prior variance $\sigma_j^2$, and prior mean zero. The posterior distributions for each variance $\sigma_j^2$ are computed, and the features whose posterior variance concentrates on small values are discarded.

### Main Reasons for Success
There are thus three main features of this approach that could be important for its success:
1. The feature selection and pre-processing,
2. the neural network model, and
3. the Bayesian inference for the model using MCMC.

#### Feature selection
According to Neal and Zhang (2006), feature screening in 1 is carried out purely for computational efficiency; the MCMC procedure is slow with a large number of features. There is no need to use feature selection to avoid overfitting. The posterior average

\begin{equation}
\text{Pr}(Y_{\text{new}} \mid X_{\text{new}}, \mathbf{X}_{\text{tr}}, \mathbf{y}_{\text{tr}}) = \int \text{Pr}( Y_{\text{new}} \mid X_{\text{new}}, \theta) \text{Pr}(\theta \mid \mathbf{X}_{\text{tr}}, \mathbf{y}_{\text{tr}}) d\theta.
\end{equation}

takes care of this automatically.

#### Effect of the Bayesian method
We would like to understand the reasons for the success of the Bayesian method. In our view, power of modern Bayesian methods does not lie in their use as a formal inference procedure; most people would not believe that the priors in a high-dimensional, complex neural network model are actually correct. Rather the Bayesian/MCMC approach gives an efficient way of sampling the relevant parts of model space, and then averaging the predictions for the high-probability models.

### Bagging and Boosting
Bagging and boosting are non-Bayesian procedure that have some similarity to MCMC in a Bayesian model. The Bayesian approach fixes the data and perturbs the parameters, according to current estimate of the posterior distribution. Bagging perturbs the data in an i.i.d. fashion and then re-estimates the model to give a new set of model parameters. At the end, a simple average of the model predictions from different bagged samples is computed.

Boosting is similar to bagging, but fits a model that is additive in the models of each individual base learner, which are learned using non i.i.d. samples. We can write all of these models in the form

\begin{equation}
\hat{f}(\mathbf{x}_{\text{new}}) = \sum_{l=1}^L w_l \text{E}(Y_{\text{new}} \mid \mathbf{x}_{\text{new}}, \hat\theta_l)
\end{equation}

In all cases the $\hat\theta_l$ are a large collection of model parameters. For the Bayesian model the $w_l = 1/L$, and the average estimates the above posterior mean by sampling $\theta_l$ from the posterior distribution. 
* For bagging, $w_l = 1/L$ as well, and the $\hat\theta_l$ are the parameters refit to bootstrap re-samples of the training data.
* For boosting, the weights are all equal to 1, but the $\hat\theta_l$ are typically chosen in a nonrandom sequential fashion to constantly improve the fit.